In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd gdrive/My\ Drive/rubik_project

/content/gdrive/My Drive/rubik_project


In [3]:
!nvidia-smi

Mon Feb 10 06:34:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import os
import numpy as np
import cv2
import PIL
import imutils

**Evaluating trained model**

In [0]:
%tensorflow_version 1.x

In [6]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [0]:
import data_provider
import config
import data_utils

In [8]:
test_data = data_provider.Dataset('public_test', config.BATCH_SIZE)

Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load emotion images.....................
Done!
Number of emotion train data:  28709
-----------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  30000
---------------------------------------------------------------
Load age image..................
Done !
Number of age train data:  42690
Number of age test data:  15280
---------------------------------------------------------------


In [9]:
import config
print(config.MODEL_DIR)

checkpoints/06022020_bn_linear


In [0]:
sess = tf.InteractiveSession()

In [0]:
import model

In [12]:
test_model = model.Multitask_BKNet(sess, False)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

(?, 24, 24, 32)
(?, 12, 12, 64)
(?, 6, 6, 128)
(?, 3, 3, 256)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
keep_dims is deprecated, use keepdims instead



Instructions for updating:
Please use tf.global_variables instead.
Reading model parameters from %s /content/gdrive/My Drive/rubik_project/checkpoints/06022020_bn_linear/model.ckpt
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/rubik_project/checkpoints/06022020_bn_linear/model.ckpt


In [13]:
test_model.count_trainable_params()

Total training params: 4.4M


In [14]:
smile_nb_true_pred = 0
emotion_nb_true_pred = 0
gender_nb_true_pred = 0
age_nb_true_pred = 0

smile_nb_test = 0
emotion_nb_test = 0
gender_nb_test = 0
age_nb_test = 0

for batch_image, batch_label, batch_index in test_data.gen():
  for i in range(len(batch_index)):
    if batch_index[i] == 1.0:
      smile_nb_test += 1
    elif batch_index[i] == 2.0:
      emotion_nb_test += 1
    elif batch_index[i] == 3.0:
      gender_nb_test += 1
    else:
      age_nb_test += 1
  
  feed_dict = {test_model.input_images: batch_image,
               test_model.input_labels: batch_label,
               test_model.input_indexes: batch_index,
               test_model.phase_train: True,
               test_model.keep_prob: 0.5}
  smile_nb_true_pred += test_model.sess.run(test_model.smile_true_pred, feed_dict=feed_dict)
  emotion_nb_true_pred += test_model.sess.run(test_model.emotion_true_pred, feed_dict=feed_dict)
  gender_nb_true_pred += test_model.sess.run(test_model.gender_true_pred, feed_dict=feed_dict)
  age_nb_true_pred += test_model.sess.run(test_model.age_true_pred, feed_dict=feed_dict)

smile_public_test_accuracy = smile_nb_true_pred * 1.0 / smile_nb_test
emotion_public_test_accuracy = emotion_nb_true_pred * 1.0 / emotion_nb_test
gender_public_test_accuracy = gender_nb_true_pred * 1.0 / gender_nb_test
age_public_test_accuracy = age_nb_true_pred * 1.0 / age_nb_test

print('\n')
print('Smile task public test accuracy: ' + str(smile_public_test_accuracy * 100))
print('Emotion task public test accuracy: ' + str(emotion_public_test_accuracy * 100))
print('Gender task public test accuracy: ' + str(gender_public_test_accuracy * 100))
print('Age task public test accuracy: ' + str(age_public_test_accuracy * 100))



Smile task public test accuracy: 86.3
Emotion task public test accuracy: 57.2025633881304
Gender task public test accuracy: 88.44099077221952
Age task public test accuracy: 52.840314136125656


In [15]:
private_data = data_provider.Dataset('private_test', config.BATCH_SIZE)

Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load emotion images.....................
Done!
Number of emotion train data:  28709
-----------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  30000
---------------------------------------------------------------
Load age image..................
Done !
Number of age train data:  42690
Number of age test data:  15280
---------------------------------------------------------------


In [16]:
smile_nb_true_pred = 0
emotion_nb_true_pred = 0
gender_nb_true_pred = 0
age_nb_true_pred = 0

smile_nb_test = 0
emotion_nb_test = 0
gender_nb_test = 0
age_nb_test = 0

for batch_image, batch_label, batch_index in private_data.gen():
  for i in range(len(batch_index)):
    if batch_index[i] == 1.0:
      smile_nb_test += 1
    elif batch_index[i] == 2.0:
      emotion_nb_test += 1
    elif batch_index[i] == 3.0:
      gender_nb_test += 1
    else:
      age_nb_test += 1
  
  feed_dict = {test_model.input_images: batch_image,
               test_model.input_labels: batch_label,
               test_model.input_indexes: batch_index,
               test_model.phase_train: True,
               test_model.keep_prob: 0.5}
  smile_nb_true_pred += test_model.sess.run(test_model.smile_true_pred, feed_dict=feed_dict)
  emotion_nb_true_pred += test_model.sess.run(test_model.emotion_true_pred, feed_dict=feed_dict)
  gender_nb_true_pred += test_model.sess.run(test_model.gender_true_pred, feed_dict=feed_dict)
  age_nb_true_pred += test_model.sess.run(test_model.age_true_pred, feed_dict=feed_dict)

smile_private_test_accuracy = smile_nb_true_pred * 1.0 / smile_nb_test
emotion_private_test_accuracy = emotion_nb_true_pred * 1.0 / emotion_nb_test
gender_private_test_accuracy = gender_nb_true_pred * 1.0 / gender_nb_test
age_private_test_accuracy = age_nb_true_pred * 1.0 / age_nb_test

print('\n')
print('Smile task private test accuracy: ' + str(smile_private_test_accuracy * 100))
print('Emotion task private test accuracy: ' + str(emotion_private_test_accuracy * 100))
print('Gender task private test accuracy: ' + str(gender_private_test_accuracy * 100))
print('Age task private test accuracy: ' + str(age_private_test_accuracy * 100))



Smile task private test accuracy: 87.0
Emotion task private test accuracy: 57.982724993034275
Gender task private test accuracy: 88.27100534239922
Age task private test accuracy: 53.062827225130896
